# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacDF = pd.read_csv('output/output.csv')
vacDF = vacDF.drop(columns = ['Unnamed: 0'])
vacDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,dossor,47.53,52.98,49.41,54,96,7.47,KZ
1,georgetown,5.41,100.34,81.28,88,20,5.82,MY
2,punta arenas,-53.15,-70.92,41.00,68,75,6.93,CL
3,castro,-24.79,-50.01,62.94,89,58,5.91,BR
4,ushuaia,-54.80,-68.30,41.00,75,75,5.82,AR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
# Use lat and lng as locations
locations = vacDF[['Lat', 'Lng']]

# Save humidity to use later as weights
humidity = vacDF['Humidity'].astype(float)

In [5]:
fig = gmaps.figure()

# Create the Heat Layer
heatLayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Change dataframe to fit weather criteria
# Criteria: Max temp between 20 and 22 degrees Celcius, wind speeds less than 10mph, and no clouds. 
hotel_df = pd.DataFrame()
hotel_df = vacDF.loc[(vacDF['Max Temp'] <= 22) & 
                       (vacDF['Max Temp'] >= 20) & 
                       (vacDF['Wind Speed'] <= 10)
                      ]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
6,yellowknife,62.46,-114.35,21.07,79,20,9.17,CA
54,sosnovo-ozerskoye,52.52,111.54,21.65,89,100,8.23,RU
96,suntar,62.14,117.63,21.47,93,59,4.34,RU
208,qaanaaq,77.48,-69.36,20.41,82,37,6.17,GL
214,seymchan,62.88,152.43,21.06,91,35,3.76,RU
238,kysyl-syr,63.90,122.76,21.81,96,96,3.11,RU
428,fairbanks,64.84,-147.72,21.60,79,75,1.12,US
497,yerbogachen,61.28,108.01,21.97,94,100,9.60,RU
513,college,64.86,-147.80,21.42,79,90,1.01,US


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
6,yellowknife,62.46,-114.35,21.07,79,20,9.17,CA
54,sosnovo-ozerskoye,52.52,111.54,21.65,89,100,8.23,RU
96,suntar,62.14,117.63,21.47,93,59,4.34,RU
208,qaanaaq,77.48,-69.36,20.41,82,37,6.17,GL
214,seymchan,62.88,152.43,21.06,91,35,3.76,RU
238,kysyl-syr,63.90,122.76,21.81,96,96,3.11,RU
428,fairbanks,64.84,-147.72,21.60,79,75,1.12,US
497,yerbogachen,61.28,108.01,21.97,94,100,9.60,RU
513,college,64.86,-147.80,21.42,79,90,1.01,US


In [25]:
# parameters to search for in google API
locations = hotel_df[["Lat", "Lng"]]

names = []

params = {
        'key': g_key,
        'radius': 5000,
        'types': 'hotel',
        'keyword': 'hotel'
    }

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = (f'{lat},{lng}')

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params = params).json()
    
    try:
        names.append(response['results'][0]['name'])
    except:
        names.append('No Hotel Found')


In [29]:
hotel_df['Hotel Name'] = names

C:\Users\Kevin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Names,Hotel Name
6,yellowknife,62.46,-114.35,21.07,79,20,9.17,CA,The Explorer Hotel,The Explorer Hotel
54,sosnovo-ozerskoye,52.52,111.54,21.65,89,100,8.23,RU,No Hotel Found,No Hotel Found
96,suntar,62.14,117.63,21.47,93,59,4.34,RU,No Hotel Found,No Hotel Found
208,qaanaaq,77.48,-69.36,20.41,82,37,6.17,GL,Qaanaaq Hotel,Qaanaaq Hotel
214,seymchan,62.88,152.43,21.06,91,35,3.76,RU,No Hotel Found,No Hotel Found
238,kysyl-syr,63.90,122.76,21.81,96,96,3.11,RU,No Hotel Found,No Hotel Found
428,fairbanks,64.84,-147.72,21.60,79,75,1.12,US,SpringHill Suites by Marriott Fairbanks,SpringHill Suites by Marriott Fairbanks
497,yerbogachen,61.28,108.01,21.97,94,100,9.60,RU,No Hotel Found,No Hotel Found
513,college,64.86,-147.80,21.42,79,90,1.01,US,SpringHill Suites by Marriott Fairbanks,SpringHill Suites by Marriott Fairbanks


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map
hotelMarkers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(hotelMarkers)
fig

Figure(layout=FigureLayout(height='420px'))